In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import scipy
import numpy as np

In [15]:
import ssvepcca
from ssvepcca.utils import load_mat_data_array
from ssvepcca.pipelines import test_fit_predict, k_fold_predict

ssvepcca.runtime_configuration.load_from_name("tsinghua-beta-dataset")

In [16]:
from ssvepcca.transformers import EEGType
from ssvepcca.algorithms import (StandardCCA, StandardCCAFilter, FilterbankCCA,
    FilterbankCCAFilter, SpatioTemporalCCA)

In [17]:
electrode_list_fbcca = ssvepcca.parameters.electrode_list_fbcca

In [18]:
import time

# Test loading data

In [19]:
mat_path = "/media/cerqueirajc/windows_heavy_data/Ubuntu/masters/dataset/tsinghua_beta_dataset/S10.mat"
# input_data = load_mat_data_array(mat_path)

In [20]:
mat = scipy.io.loadmat(mat_path)

In [21]:
input_data = mat["data"][0][0][0]
input_data = input_data.astype(float).T.transpose([1, 0, 2, 3])

In [22]:
input_data.shape

(4, 40, 750, 64)

In [44]:
len(mat["data"][0][0])

2

In [45]:
mat["data"][0][0][0].shape[1] / 250

4.0

In [46]:
electrodes_info = mat["data"][0][0][1][0][0][3]

In [47]:
pd.DataFrame(electrodes_info)

,0,1,2,3
0,[1],[-17.926],[0.51499],[FP1]
1,[2],[0],[0.50669],[FPZ]
2,[3],[17.926],[0.51499],[FP2]
3,[4],[-22.461],[0.42113],[AF3]
4,[5],[22.461],[0.42113],[AF4]
...,...,...,...,...
59,[60],[0],[0],[CB1]
60,[61],[-162.07],[0.51499],[O1]
61,[62],[180],[0.50669],[OZ]
62,[63],[162.07],[0.51499],[O2]


In [48]:
for i in range(64):
    # print(electrodes_info[i][0][0], electrodes_info[i][3][0])
    print(f'"{electrodes_info[i][3][0]}": {int(electrodes_info[i][0][0]) - 1},')

"FP1": 0,
"FPZ": 1,
"FP2": 2,
"AF3": 3,
"AF4": 4,
"F7": 5,
"F5": 6,
"F3": 7,
"F1": 8,
"FZ": 9,
"F2": 10,
"F4": 11,
"F6": 12,
"F8": 13,
"FT7": 14,
"FC5": 15,
"FC3": 16,
"FC1": 17,
"FCZ": 18,
"FC2": 19,
"FC4": 20,
"FC6": 21,
"FT8": 22,
"T7": 23,
"C5": 24,
"C3": 25,
"C1": 26,
"CZ": 27,
"C2": 28,
"C4": 29,
"C6": 30,
"T8": 31,
"M1": 32,
"TP7": 33,
"CP5": 34,
"CP3": 35,
"CP1": 36,
"CPZ": 37,
"CP2": 38,
"CP4": 39,
"CP6": 40,
"TP8": 41,
"M2": 42,
"P7": 43,
"P5": 44,
"P3": 45,
"P1": 46,
"PZ": 47,
"P2": 48,
"P4": 49,
"P6": 50,
"P8": 51,
"PO7": 52,
"PO5": 53,
"PO3": 54,
"POZ": 55,
"PO4": 56,
"PO6": 57,
"PO8": 58,
"CB1": 59,
"O1": 60,
"OZ": 61,
"O2": 62,
"CB2": 63,


In [49]:
electrodes_info[2][0][0]

'3'

In [51]:
pd.DataFrame(mat["data"][0][0][1][0])

,sub,age,gender,chan,freqs,phases,bci_quotient,wide_snr,narrow_snr,srate
0,[S30],[[22.25]],[female],"[[[1], [-17.926], [0.51499], [FP1]], [[2], [0]...","[[8.6, 8.799999999999999, 9.0, 9.2, 9.4, 9.6, ...","[[4.71238898038469, 0.0, 1.5707963267948966, 3...",[[114.19696894054809]],[[-11.593666783155593]],[[7.229504071445169]],[[250]]


In [53]:
pd.DataFrame(mat["data"][0][0][1][0])["freqs"].loc[0][0]

array([ 8.6,  8.8,  9. ,  9.2,  9.4,  9.6,  9.8, 10. , 10.2, 10.4, 10.6,
       10.8, 11. , 11.2, 11.4, 11.6, 11.8, 12. , 12.2, 12.4, 12.6, 12.8,
       13. , 13.2, 13.4, 13.6, 13.8, 14. , 14.2, 14.4, 14.6, 14.8, 15. ,
       15.2, 15.4, 15.6, 15.8,  8. ,  8.2,  8.4])

In [54]:
pd.DataFrame(mat["data"][0][0][1][0])["phases"].loc[0][0]

array([4.71238898, 0.        , 1.57079633, 3.14159265, 4.71238898,
       0.        , 1.57079633, 3.14159265, 4.71238898, 0.        ,
       1.57079633, 3.14159265, 4.71238898, 0.        , 1.57079633,
       3.14159265, 4.71238898, 0.        , 1.57079633, 3.14159265,
       4.71238898, 0.        , 1.57079633, 3.14159265, 4.71238898,
       0.        , 1.57079633, 3.14159265, 4.71238898, 0.        ,
       1.57079633, 3.14159265, 4.71238898, 0.        , 1.57079633,
       3.14159265, 4.71238898, 0.        , 1.57079633, 3.14159265])

# Test algorithm

In [26]:
start_time_index = round(125 + 250 * 0.130)
window_size_in_seconds = 3
stop_time_index = 125 * (2 * window_size_in_seconds + 1)
(start_time_index, stop_time_index)

(158, 875)

In [27]:
result_fbcca_single_component = test_fit_predict(
    input_data, 
    FilterbankCCA(
        electrodes_name=electrode_list_fbcca,
        start_time_idx=start_time_index,
        stop_time_idx=stop_time_index,
        num_harmonics=5,
        fb_num_subband=10,
        fb_fundamental_freq=8,
        fb_upper_bound_freq=88,
    )
)

In [28]:
result_fbcca_single_component[2]

[122, 0.7625]

In [29]:
result_fbcca_single_component[0].shape

(4, 40)

In [30]:
ground_truth = np.array([i for i in range(40)])[np.newaxis, :].repeat(4, axis=0)
pd.DataFrame((result_fbcca_single_component[0] == ground_truth).sum(axis=0))

,0
0,3
1,0
2,0
3,0
4,4
5,0
6,0
7,0
8,4
9,1
